<a href="https://colab.research.google.com/github/CAIOZANETTI/eng_civil_calc/blob/main/silo_vertical_soja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import math
import pandas as pd


# -----------------------------------------------------------
#   FUNÇÕES AUXILIARES
# -----------------------------------------------------------
def p_lateral(z, k=0.4, gamma=8.0):
    return k * gamma * z               # kN/m²


def hoop_stress(p, r, t):
    return p * r / t / 1_000           # MPa


def forca_anel(z_inf, z_sup, r, t, k=0.4, gamma=8.0):
    Δz = z_sup - z_inf
    z_mid = 0.5 * (z_inf + z_sup)
    p_mid = p_lateral(z_mid, k, gamma)
    σθ = hoop_stress(p_mid, r, t)
    F_anel = σθ * t * 1_000 * Δz       # kN
    return F_anel, σθ, p_mid


# -----------------------------------------------------------
#   FUNÇÃO PRINCIPAL
# -----------------------------------------------------------
def calcular_silo_protendido(
    D, h, t, fck,
    *,
    gamma_soja=8, k=0.4,
    fpk=1_860, σp_ef=1_020,       # MPa
    A_cabo_mm2=280,
    dens_aco=7_850,
    espaç=0.5,
    fator=1.30,
    ρ_v=0.003,                    # fração da seção de parede
    φ_v_mm=16,                    # Ø barras verticais
    preco_conc=800,               # R$/m³
    preco_aco_pass=12,            # R$/kg
    preco_aco_ativo=15,           # R$/kg
    interativo=True,
):
    """
    Dimensiona silo protendido (cordoalhas + passiva vertical) e estima custo.
    Retorno:
        - interativo=True  ➜  (df_faixas, df_passiva, df_resumo_custos)
        - interativo=False ➜  df_resumo_custos
    """

    # ---------------- Validações básicas ----------------
    if σp_ef > 0.55 * fpk:
        raise ValueError("σ_p,ef excede 0,55·f_pk")
    if any(x <= 0 for x in (D, h, t, espaç)):
        raise ValueError("Dimensões devem ser positivas.")

    # ---------------- Geometria & constantes ------------
    r = D / 2
    circ = 2 * math.pi * r
    A_cabo_m2 = A_cabo_mm2 / 1e6
    F_cabo = σp_ef * 1_000 * A_cabo_m2          # kN

    # ----------------------------------------------------
    #   (1) CORDOALHAS – faixa a faixa
    # ----------------------------------------------------
    faixas, total_cabos = [], 0
    n_faixas = math.ceil(h / espaç)

    for i in range(n_faixas):
        z_inf = i * espaç
        z_sup = min(h, (i + 1) * espaç)
        F_anel, σθ, p_mid = forca_anel(z_inf, z_sup, r, t, k, gamma_soja)
        F_req = fator * F_anel
        n_cabos = max(1, math.ceil(F_req / F_cabo))

        faixas.append(
            dict(faixa=i + 1, z_inf=round(z_inf, 2), z_sup=round(z_sup, 2),
                 p_med=round(p_mid, 2), σθ_med=round(σθ, 2),
                 F_anel_kN=round(F_anel, 1), cabos=n_cabos)
        )
        total_cabos += n_cabos

    df_faixas = pd.DataFrame(faixas)

    L_cabos = total_cabos * circ
    V_aco_ativo = L_cabos * A_cabo_m2
    m_aco_ativo = V_aco_ativo * dens_aco

    # ----------------------------------------------------
    #   (2) ARMADURA PASSIVA VERTICAL
    # ----------------------------------------------------
    A_bar_mm2 = math.pi * (φ_v_mm ** 2) / 4
    A_bar_m2 = A_bar_mm2 / 1e6

    As_min_m2_m = ρ_v * t
    barras_por_m = math.ceil(As_min_m2_m / A_bar_m2)
    espaçamento_mm = 1_000 / barras_por_m         # mm

    n_barras_circ = barras_por_m * circ
    L_barras = n_barras_circ * h
    V_aco_pass = L_barras * A_bar_m2
    m_aco_pass = V_aco_pass * dens_aco

    df_passiva = pd.DataFrame([{
        "φ_mm": φ_v_mm,
        "ρ_v": ρ_v,
        "barras_por_m": barras_por_m,
        "espaçamento_mm": round(espaçamento_mm, 1),
        "n_barras_totais": round(n_barras_circ),
        "massa_aco_passivo_kg": round(m_aco_pass, 1),
    }])

    # ----------------------------------------------------
    #   (3) CONCRETO
    # ----------------------------------------------------
    V_conc = circ * h * t

    # ----------------------------------------------------
    #   (4) CUSTOS
    # ----------------------------------------------------
    custo_conc = V_conc * preco_conc
    custo_ativo = m_aco_ativo * preco_aco_ativo
    custo_passivo = m_aco_pass * preco_aco_pass
    custo_total = custo_conc + custo_ativo + custo_passivo

    índice_aco = (m_aco_ativo + m_aco_pass) / V_conc

    df_resumo = pd.DataFrame([{
        "D (m)": D,
        "h (m)": h,
        "t (m)": t,
        "V_concreto_m3": round(V_conc, 2),
        "massa_ativo_kg": round(m_aco_ativo, 1),
        "massa_passivo_kg": round(m_aco_pass, 1),
        "índice_aco_kg_m3": round(índice_aco, 1),
        "custo_concreto_R$": round(custo_conc, 2),
        "custo_ativo_R$": round(custo_ativo, 2),
        "custo_passivo_R$": round(custo_passivo, 2),
        "custo_total_R$": round(custo_total, 2),
        "total_cabos": total_cabos,
    }])

    # ---------- retorno ----------
    if interativo:
        return df_faixas, df_passiva, df_resumo
    else:
        return df_resumo


# -----------------------------------------------------------
#   EXEMPLO RÁPIDO DE USO
# -----------------------------------------------------------
if __name__ == "__main__":
    # interativo=False devolve só o DataFrame de resumo
    resumo = calcular_silo_protendido(30, 48, 0.25, 40, interativo=True)
    print(resumo.to_string(index=False))


AttributeError: 'tuple' object has no attribute 'to_string'

In [13]:
import numpy as np

D_alternativas = np.arange(24, 36, 2)     # diâmetros de 24 a 34 m
h_alternativas = np.arange(35, 55, 5)     # alturas de 35 a 50 m
t_alternativas = [0.20, 0.25, 0.30]       # espessuras

resultados = []

# Volume‑alvo:  (π·D²/4)·h  ↦ mantenha aproximadamente igual
V_alvo = (math.pi * 30**2 / 4) * 48      # mesmo silo‑exemplo

for D in D_alternativas:
    for h in h_alternativas:
        V_bruto = (math.pi * D**2 / 4) * h
        if abs(V_bruto - V_alvo) / V_alvo > 0.05:   # ±5 % de tolerância
            continue

        for t in t_alternativas:
            df = calcular_silo_protendido(D, h, t, 40, interativo=False)
            resultados.append(df.iloc[0])           # pega linha única

df_res = pd.DataFrame(resultados).sort_values("custo_total_R$")
print(df_res.head(10).to_string(index=False))


TypeError: calcular_silo_protendido() got an unexpected keyword argument 'interativo'

In [ ]:
faixas, passiva, resumo = calcular_silo_protendido(30, 48, 0.40, 40)
display(faixas)
display(passiva)
display(resumo)